In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def parse_mapping_xml(mapping_file):
    try:
        tree = ET.parse(mapping_file)
        root = tree.getroot()
        
        sources = {}
        targets = {}
        transformations = []

        # Extract source information
        for source in root.findall(".//SOURCE"):
            source_name = source.get('NAME')
            if source_name not in sources:
                sources[source_name] = []
            for field in source.findall(".//SOURCEFIELD"):
                field_name = field.get('NAME')
                sources[source_name].append(field_name)

        # Extract target information
        for target in root.findall(".//TARGET"):
            target_name = target.get('NAME')
            if target_name not in targets:
                targets[target_name] = []
            for field in target.findall(".//TARGETFIELD"):
                field_name = field.get('NAME')
                targets[target_name].append(field_name)

        # Extract transformation information
        for transformation in root.findall(".//TRANSFORMATION"):
            transformation_name = transformation.get('NAME')
            transformation_type = transformation.get('TYPE')
            for field in transformation.findall(".//TRANSFORMFIELD"):
                field_name = field.get('NAME')
                port_type = field.get('PORTTYPE')
                transformations.append({
                    'Transformation Name': transformation_name,
                    'Transformation Type': transformation_type,
                    'Field Name': field_name,
                    'Port Type': port_type
                })

        return sources, targets, transformations
    except Exception as e:
        print(f"Error parsing mapping XML: {e}")
        return {}, {}, []

def create_data_lineage(sources, targets, transformations):
    data_lineage = []

    # Create a reverse mapping of source fields to source names
    source_fields_mapping = {}
    for source_name, fields in sources.items():
        for field in fields:
            source_fields_mapping[field] = source_name

    for target_name, target_fields in targets.items():
        for target_field in target_fields:
            lineage_entry = {
                'Source Name': '',
                'Source Field': '',
                'Transformation Name': '',
                'Transformation Type': '',
                'Target Name': target_name,
                'Target Field': target_field
            }

            # First try to find through transformations
            transformation_found = False
            for transformation in transformations:
                if transformation['Port Type'] == 'OUTPUT' and transformation['Field Name'] == target_field:
                    lineage_entry['Transformation Name'] = transformation['Transformation Name']
                    lineage_entry['Transformation Type'] = transformation['Transformation Type']
                    
                    # Try to find source from transformation input
                    if transformation['Field Name'] in source_fields_mapping:
                        lineage_entry['Source Name'] = source_fields_mapping[transformation['Field Name']]
                        lineage_entry['Source Field'] = transformation['Field Name']
                        transformation_found = True
                        break

            # If no transformation found, try direct target-to-source match
            if not transformation_found:
                # Check if target field exists in any source
                if target_field in source_fields_mapping:
                    lineage_entry['Source Name'] = source_fields_mapping[target_field]
                    lineage_entry['Source Field'] = target_field
                # If not found, mark as direct target (no source)
                else:
                    lineage_entry['Source Name'] = 'Direct Target'
                    lineage_entry['Source Field'] = target_field

            data_lineage.append(lineage_entry)

    return data_lineage


# for saving the file in excel
def save_to_excel(data_lineage, output_file):
    try:
        df = pd.DataFrame(data_lineage)
        
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Data Lineage', index=False)
        
        print(f"Data lineage has been successfully written to {output_file}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")

# File paths
mapping_file = 'Python_tutorials_mapping.XML'
output_file = 'data_lineage222.xlsx'

# Parse mapping file
sources, targets, transformations = parse_mapping_xml(mapping_file)

# Create data lineage
data_lineage = create_data_lineage(sources, targets, transformations)

# Save to Excel
save_to_excel(data_lineage, output_file)